In [83]:
import pandas as pd
import re
import numpy as np
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from numpy import asarray
from numpy import array
from numpy import zeros
from typing import List

[nltk_data] Downloading package punkt to /Users/sherryliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('tinder_google_play_reviews.csv')

In [4]:
df.shape

(535406, 10)

In [5]:
df.dropna(subset=['content'],inplace=True)
df.drop(columns=['reviewId', 'userImage'],inplace=True)
df.shape

(534062, 8)

In [6]:
df.sort_values('at', ascending=False, inplace=True)

In [7]:
df = df.head(100000)
df.tail(5)

,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
99997,Daniel Rothermel,✓✓✓✓,5,0,11.23.0,2020-09-17 00:35:01,NaN,NaN
99998,Guero Guero,Mature grown with experience,5,0,NaN,2020-09-17 00:06:21,NaN,NaN
99999,Alexander Waalen,I changed my phone # and forgot to cancel my s...,1,0,NaN,2020-09-16 23:49:52,NaN,NaN
100000,Ethan Shadows,My account was banned for reasons unbeknownst ...,1,1,11.23.0,2020-09-16 23:44:10,NaN,NaN
100001,Stacey K,Not fun and have to pay for everything.,1,0,NaN,2020-09-16 23:32:37,NaN,NaN


In [8]:
df.isna().mean()

userName                0.00003
content                 0.00000
score                   0.00000
thumbsUpCount           0.00000
reviewCreatedVersion    0.26611
at                      0.00000
replyContent            0.99574
repliedAt               0.99574
dtype: float64

In [66]:
stopword_list = stopwords.words("english")

def get_vec(words, stop=stopword_list):
    '''Revise the function by changing stop_words and adding token_pattern'''
    vectorizer = CountVectorizer(stop_words=stop, lowercase=True, min_df=0.001, token_pattern=r'[a-z\_\-]{3,}', binary=True)
    X = vectorizer.fit_transform(words) 
    X = X.toarray()
    print(X.shape)
    feature = vectorizer.get_feature_names()
    corpus_df = pd.DataFrame(X, columns=feature)
    return corpus_df

In [10]:
review = df['content'].tolist()
review = [line.lower() for line in review]

In [11]:
corpus_df = get_vec(review, 'english')
corpus_df.sum().sort_values(ascending=False)[:50]

(100000, 1050)


app             31111
tinder          11476
banned          10966
good            10184
people           9783
just             9090
account          8509
don              7392
like             7219
money            6971
time             6434
reason           6328
got              6099
pay              5715
matches          5525
fake             4930
use              4810
match            4712
profiles         4248
profile          4069
nice             4032
great            3828
likes            3741
dating           3675
waste            3501
new              3398
gold             3296
want             3290
bad              2988
worst            2951
way              2934
subscription     2860
make             2811
know             2682
really           2671
doesn            2585
paid             2546
used             2492
getting          2450
using            2395
free             2340
love             2293
better           2265
didn             2194
service          2179
work      

In [12]:
# https://tinder.com/feature/subscription-tiers
def word_replace_pre_lem(line):
    '''Before lemmitization, do the regex cleaning for features in Tinder'''
    line = re.sub(r'\bwho likes? you\b', '_who_like_you_', line)
    line = re.sub(r'\bsuper likes?\b', '_super_like_', line)
    line = re.sub(r'\blike(s|d)?\b', '_likes_', line)
    line = re.sub(r'\bmatch(es|ed)?\b', '_matches_', line)
    line = re.sub(r'\brewind(s)?\b', '_rewind_', line)
    line = re.sub(r'\bpassport\b', '_passport_', line)
    line = re.sub(r'\bmessages?\b', '_message_', line)
    line = re.sub(r'\bmeets?\b', '_meet_', line)
    line = re.sub(r'\bswipe(s| left| right)?\b', '_swipe_', line)
    line = re.sub(r'\btop picks?\b', '_top_pick_', line)
    line = re.sub(r'\b(tinder )?gold\b', '_tinder_gold_', line)
    line = re.sub(r'\b(tinder )?plus\b', '_tinder_plus_', line)
    line = re.sub(r'\b(tinder )?platinum\b', '_tinder_platinum_', line)
    line = re.sub(r'\bpremium( features?)?\b', '_premium_', line)
    return line

In [13]:
review_pre_lem = [word_replace_pre_lem(line) for line in review]

In [14]:
corpus_df = get_vec(review_pre_lem, 'english')
corpus_df.sum().sort_values(ascending=False)[:50]

(100000, 1046)


app              31111
banned           10966
good             10184
tinder           10176
_matches_        10060
people            9783
_likes_           9571
just              9090
account           8509
don               7392
money             6971
time              6434
reason            6328
got               6099
pay               5715
fake              4930
use               4810
profiles          4248
profile           4069
nice              4032
great             3828
dating            3675
waste             3501
_message_         3479
new               3398
_tinder_gold_     3296
want              3290
bad               2988
worst             2951
way               2934
subscription      2860
make              2811
know              2682
really            2671
doesn             2585
paid              2546
used              2492
getting           2450
using             2395
free              2340
love              2293
better            2265
didn              2194
service    

In [15]:
## Reference: https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

def lem(sentence):
    '''Intake a list of review, lemmatize them and return a new list'''
    result_lem = []
    for s in tqdm(sentence):
        s_lem = lemmatize_sentence(s)
        result_lem.append(s_lem)
    return result_lem

In [65]:
def get_vec(words, stop=stopword_list):
    '''Revise the function by changing stop_words and adding token_pattern'''
    vectorizer = CountVectorizer(stop_words=stop, lowercase=True, min_df=0.001, token_pattern=r'[a-z\_\-]{3,}', binary=True)
    X = vectorizer.fit_transform(words) 
    X = X.toarray()
    print(X.shape)
    feature = vectorizer.get_feature_names()
    corpus_df = pd.DataFrame(X, columns=feature)
    return corpus_df

In [48]:
review_lem = lem(review_pre_lem)
corpus_df = get_vec(review_lem, 'english')
corpus_df.sum().sort_values(ascending=False)[:50]

100%|█████████████████████████████████| 100000/100000 [01:17<00:00, 1287.17it/s]


(100000, 860)


app              31112
ban              12267
good             11529
tinder           10220
_matches_        10060
account           9971
people            9887
_likes_           9571
pay               9129
just              9090
use               9054
profile           7785
time              7765
money             6974
reason            6631
bad               6053
make              5420
fake              5178
try               5041
work              4534
say               4161
want              4052
nice              4038
waste             3967
great             3861
date              3609
_message_         3479
new               3439
_tinder_gold_     3296
day               3188
subscription      3068
way               3065
know              3037
month             2774
year              2703
really            2671
need              2665
service           2577
look              2504
love              2479
delete            2471
free              2342
experience        2292
send       

In [49]:
corpus_df = get_vec(review_lem, stopword_list)
corpus_df.sum().sort_values(ascending=False)[:50]

(100000, 964)


get              16019
ban              12267
good             11529
tinder           10220
_matches_        10060
account           9971
_likes_           9571
pay               9129
use               9054
even              8058
profile           7785
time              7765
money             6974
reason            6631
bad               6053
make              5420
fake              5178
see               5161
try               5041
one               5008
work              4534
say               4161
give              4120
want              4052
nice              4038
waste             3967
great             3861
find              3743
date              3609
show              3601
_message_         3479
new               3439
keep              3397
never             3393
_tinder_gold_     3296
still             3276
many              3264
would             3258
day               3188
nothing           3154
subscription      3068
way               3065
know              3037
without    

In [50]:
def word_replace_post_lem(line):
    '''After lemmitization, do the regex cleaning for known product issue'''
    line = re.sub(r'\b(ban|block|close|suspend)\b', '_ban_', line)
    line = re.sub(r'\b(bug|crash)\b', '_bug_', line)
    line = re.sub(r'\blog(-?in)?\b', '_login_', line)
    line = re.sub(r'\b(fake( account| profile)?|bot|spam)\b', '_fake_account_', line)
    line = re.sub(r'\b(scam|cheat)\b', '_scam_', line)
    line = re.sub(r'\b(subscribe|subscription|membership)\b', '_subscription_', line)
    line = re.sub(r'\b(charge|paywall|payment|in-app-purchase)\b', '_payment_', line)
    return line

In [51]:
review_final = [word_replace_post_lem(review) for review in review_lem]
corpus_df = get_vec(review_final, stopword_list)
corpus_df.sum().sort_values(ascending=False)[:50]

(100000, 953)


get               16019
_ban_             13797
good              11529
tinder            10220
_matches_         10060
_likes_            9571
account            9245
pay                9129
use                9054
even               8058
time               7765
_fake_account_     7026
money              6974
reason             6631
profile            6208
bad                6053
make               5420
see                5161
try                5041
one                5008
work               4534
_subscription_     4441
say                4161
give               4120
want               4052
nice               4038
waste              3967
great              3861
find               3743
date               3609
show               3601
_message_          3479
new                3439
keep               3397
never              3393
_tinder_gold_      3296
still              3276
many               3264
would              3258
day                3188
nothing            3154
way             

In [52]:
df['review_final'] = review_final

In [64]:
review_good = df[df['score'] >= 4]['review_final']
review_bad = df[df['score'] < 4]['review_final']

In [86]:
stopword_list.append('app')

def get_tfidf_vec(review):
    '''Intake a list of review and return tf-idf report'''
    vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z\_]{3,}\b',
                             max_df=0.4, stop_words=stopword_list, max_features=1000, binary=True)
    X = vectorizer.fit_transform(review)
    terms = vectorizer.get_feature_names()
    tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
    tf_idf_sum = tf_idf.sum(axis=1)
    score = pd.DataFrame(tf_idf_sum, columns=["score"])
    score.sort_values(by="score", ascending=False, inplace=True)
    nmf = NMF(n_components=4)
    W = nmf.fit_transform(X)
    H = nmf.components_
    tf_idf_topic = pd.DataFrame(X.toarray(), columns=terms)
    def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
        for topic, vector in enumerate(H):
            print(f"TOPIC {topic}\n")
            total = vector.sum()
            top_scores = vector.argsort()[::-1][:num_top_tokens]
            token_names = list(map(lambda idx: feature_names[idx], top_scores))
            strengths = list(map(lambda idx: vector[idx] / total, top_scores))

            for strength, token_name in zip(strengths, token_names):
                print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
            print(f"=" * 50)
    get_top_tf_idf_tokens_for_topic(H, tf_idf_topic.columns.tolist(), 5)
    return score

In [87]:
score_good = get_tfidf_vec(review_good)
score_good[:20]

/Users/sherryliu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


TOPIC 0

easy use (56.4%)

good easy (3.4%)

nice easy (3.0%)

great easy (2.5%)

fun easy (1.3%)

TOPIC 1

far good (76.5%)

good still (0.8%)

good meet (0.8%)

good really (0.8%)

good experience (0.7%)

TOPIC 2

new people (18.7%)

_meet_ new (15.4%)

meet new (5.0%)

way _meet_ (3.7%)

new friend (3.1%)

TOPIC 3

best ever (69.1%)

best dating (7.8%)

one best (1.9%)

ever see (1.6%)

tinder best (1.0%)



/Users/sherryliu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


,score
easy use,264.285630
far good,162.690835
new people,152.227050
best dating,145.555727
best ever,128.800345
pretty good,121.157122
_meet_ new,119.238975
best date,115.329946
get _matches_,113.456256
_meet_ people,100.609507


In [88]:
score_bad = get_tfidf_vec(review_bad)
score_bad[:20]

/Users/sherryliu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


TOPIC 0

_ban_ reason (62.3%)

got _ban_ (7.5%)

reason even (1.3%)

reason give (1.0%)

reason explanation (0.8%)

TOPIC 1

waste time (38.2%)

time money (5.1%)

get _matches_ (1.5%)

dont waste (1.3%)

complete waste (1.0%)

TOPIC 2

get _ban_ (26.2%)

account get (5.5%)

_ban_ without (1.5%)

_ban_ nothing (1.1%)

customer service (0.9%)

TOPIC 3

account _ban_ (21.7%)

_ban_ account (2.8%)

_ban_ without (2.3%)

say account (1.9%)

without reason (1.5%)



,score
_ban_ reason,1504.349171
get _ban_,1244.846308
waste time,1170.816817
account _ban_,841.360155
get _matches_,756.659753
_ban_ account,586.002440
many _fake_account_,484.777878
delete account,455.150144
customer service,438.820651
waste money,432.051418
